In [1]:
# Goal: compare qiskit aer simulator with the cirq one

In [79]:
import numpy as np
import cirq
import qiskit
from qiskit.circuit import qpy_serialization
from qiskit import QuantumCircuit
from typing import List
import math
import pandas as pd
from IPython.display import display
from tqdm import tqdm

In [3]:
# N QUBITS = 10

# Randomly generate an encoding for a circit

    # Preselection
    # IDEA: maybe we should keep only programs which output a subset of states and not superpositions
    # https://quantumcomputing.stackexchange.com/a/14398/18188
    # how-many-shots-should-one-take-to-get-a-reliable-estimate-in-a-quantum-program

# Convert that encoding to a qasm string

# for each platform in (qiskit, cirq, etc)

    # create a circuit object form the qasm file
    
    # perform 30 evaluations [N_EVALUATION]
        
        # run the circuit for 1024 or 8096 shots
        
    # save the statistics for each combination to a dataframe
    
# compare the distribution of the occurrences of each basis state (e.g. 0000)
# for cirq (state: 0000): [512, 493, 502, etc ... until N_EVALUATION]
# for qiskit (state: 0000): [400, 475, 480, etc ... until N_EVALUATION]
# check if the two sequences come from the same distribution with a test
from scipy.stats import ttest_ind
data1 = [0.873, 2.817, 0.121, -0.945, -0.055, -1.436, 0.360, -1.478, -1.637, -1.869]
data2 = [1.142, -0.432, -0.938, -0.729, -0.846, -0.157, 0.500, 1.183, -1.075, -0.169]
stat, p = ttest_ind(data1, data2)

# if yes... then bug discovered!

# next steps:
    
    # which platform is buggy: cirq or qiskit?
    
    # where was the mistake in that platform?
    
        # delta debugging (style)
        
        # - remove operations
        # - track the function called with cProfile
        # - continue until the removed operation cause the Cirq an Qiskit result 
        # to agree again

In [86]:
N_QUBITS = 10
N_OPS = 100
N_EVALUATIONS = 100
TOP_K_SOLUTION_TO_COMPARE = 5
CLASSICAL_INPUT = 127
SEED_NP = 42

QASM_HEADER = """
OPENQASM 2.0;
include "qelib1.inc";
"""

gate_set = {
   "cx": 3, 
    "rx": 1, 
    "ry": 1, 
    "rz": 1, 
    "p": 1
}

In [40]:
# Randomly generate an encoding for a circit
def random_circuit_encoding(n_ops=N_OPS, random_state=None):
    if random_state is not None:
        assert isinstance(random_state, int)
        np.random.seed(random_state)
    return np.random.rand(3 * n_ops)

random_circuit_encoding(random_state=42)

array([0.37454012, 0.95071431, 0.73199394, 0.59865848, 0.15601864,
       0.15599452, 0.05808361, 0.86617615, 0.60111501, 0.70807258,
       0.02058449, 0.96990985, 0.83244264, 0.21233911, 0.18182497,
       0.18340451, 0.30424224, 0.52475643, 0.43194502, 0.29122914,
       0.61185289, 0.13949386, 0.29214465, 0.36636184, 0.45606998,
       0.78517596, 0.19967378, 0.51423444, 0.59241457, 0.04645041,
       0.60754485, 0.17052412, 0.06505159, 0.94888554, 0.96563203,
       0.80839735, 0.30461377, 0.09767211, 0.68423303, 0.44015249,
       0.12203823, 0.49517691, 0.03438852, 0.9093204 , 0.25877998,
       0.66252228, 0.31171108, 0.52006802, 0.54671028, 0.18485446,
       0.96958463, 0.77513282, 0.93949894, 0.89482735, 0.59789998,
       0.92187424, 0.0884925 , 0.19598286, 0.04522729, 0.32533033,
       0.38867729, 0.27134903, 0.82873751, 0.35675333, 0.28093451,
       0.54269608, 0.14092422, 0.80219698, 0.07455064, 0.98688694,
       0.77224477, 0.19871568, 0.00552212, 0.81546143, 0.70685

In [41]:
def slot_to_gate(gates, gate_weights):
    mapping_dict = dict()
    slot_to_assign = 0
    for gate, number_of_slots in zip(gates, gate_weights):
        for slot in range(number_of_slots):
            mapping_dict[slot_to_assign] = gate
            slot_to_assign += 1
    return mapping_dict


def param_to_gate(param, gate_set):
    map_slot_to_gate = slot_to_gate(gate_set.keys(), gate_set.values())
    slots = sum(gate_set.values())
    op_type = map_slot_to_gate[int(param / (float(1) / slots))]
    return op_type

#def param_to_qubit(param, available_indices):
    

In [99]:
# Convert that encoding to a qasm string
def encoding_to_circuit(encoding: List[float], n_qubits: int, classical_input: int = None):
    """Convert encoding to a qasm string.
    
    We have a list to encode the single operation, the final list looks like:
    (0.23, 0.75, 0.76, 0.44, etc)
    (gate_type, qubit, parameter, gate_type, qubit, parameter, etc...)
    The single operation is made of three floats: 
    (gate_type, qubit, parameter)
    The first float encodes the gate. We divide the interval uniformly 
    on the available gates:
    - CNOT
    - pauli X
    - pauli Y
    - pauli Z
    - pahse shift
    The second float encodes the qubit on which it applies to. We divide the
    interval uniformly on the available qubits.
    The third float is used only by:
    - cnot gate: to select a target qubit (similarly to the first flaot)
    - phase shift: to select the rotation
    Based on the second digit we decide on which qubit it acts on
    
    """
    circuit_qasm = QASM_HEADER
    qubits = range(n_qubits)
    
    # add quantum and classical registers
    circuit_qasm += f"qreg q[{n_qubits}];\n"
    circuit_qasm += f"creg c[{n_qubits}];\n"
    
    
    if classical_input != None:
        assert isinstance(classical_input, int)
        format_string = "{0:" + str(n_qubits) + "b}"
        string_input = format_string.format(classical_input).zfill(n_qubits)
        for i, c in enumerate(string_input):
            if c == "1": 
                circuit_qasm += f"x q[{i}];\n"
    
    circuit_qasm += f"barrier q;\n"
    
    
    # get the single chunks
    n = 3  # number of parameters per operation
    chunks = [encoding[i:i + n] for i in range(0, len(encoding), n)]
    for op in chunks:
        # discard incomplete sequences
        if len(op) != 3: 
            continue
        # get the type of gate
        op_type = param_to_gate(param=op[0], gate_set=gate_set)
        #print(op_type, " : ", op[0])        
        
        try:
            # get target qubit
            qubit = qubits[int(op[1] / (float(1) / (len(qubits))))]
            #print(qubit, " : ", op[1])

            # extra parameter
            if op_type == "cx":
                # get second target qubit
                index_target = int(op[2] / (float(1) / (len(qubits) - 1)))
                if index_target >= qubit:
                    index_target += 1
                second_target_qubit = qubits[index_target]
                #print(second_target_qubit, " : ", op[2])
                #assert qubit == second_target_qubit, "Invalid encoding CNOT gate with same control and target qubit."
                circuit_qasm += f"cx q[{qubit}], q[{second_target_qubit}];\n"
            elif op_type == "p":
                # get rotation parameter
                parameter = 2 * math.pi * op[2]
                #print(parameter, " : ", op[2])
                circuit_qasm += f"U(0,{parameter},0) q[{qubit}];\n"
            else:
                parameter = 2 * math.pi * op[2]
                # call the simple X, Y, Z gate on a single qubit
                circuit_qasm += f"{op_type}({parameter}) q[{qubit}];\n"
        
        except IndexError: 
            print(f"op[0]: {op[0]}")
            print(f"op[1]: {op[1]}")
            print(f"op[2]: {op[2]}")
            print(f"op_type: {op_type}")
            print(f"qubit: {qubit}")
            print(f"second_target_qubit: {second_target_qubit}")
    circuit_qasm += f"barrier q;\n"
    # Measure
    circuit_qasm += f"measure q -> c;\n"
    return circuit_qasm

random_circuit_qasm_str = encoding_to_circuit(
        encoding=random_circuit_encoding(), 
        n_qubits=10, 
        classical_input=30)
print(random_circuit_qasm_str)
random_circuit = QuantumCircuit.from_qasm_str(random_circuit_qasm_str)
random_circuit.draw()


OPENQASM 2.0;
include "qelib1.inc";
qreg q[10];
creg c[10];
x q[5];
x q[6];
x q[7];
x q[8];
barrier q;
rx(4.549443067619065) q[9];
cx q[4], q[7];
cx q[2], q[8];
rz(0.5848724731286916) q[1];
rx(1.018671995531665) q[4];
rz(5.729270117646445) q[9];
rz(0.5910009196472176) q[6];
rz(2.287387844749775) q[0];
cx q[4], q[9];
cx q[3], q[0];
rx(4.090920713503318) q[2];
U(0,2.448511823148002,0) q[4];
cx q[3], q[8];
rz(6.142850641085208) q[5];
cx q[2], q[0];
U(0,1.722863917757805,0) q[3];
cx q[7], q[9];
rx(6.206264552450614) q[2];
cx q[6], q[0];
U(0,5.953323295445003,0) q[1];
U(0,1.7731289219814756,0) q[1];
ry(2.336492374389021) q[1];
cx q[2], q[5];
cx q[9], q[4];
U(0,1.567379940528729,0) q[2];
U(0,1.2855046375973904,0) q[0];
U(0,1.7410279940484688,0) q[0];
U(0,2.323940517329929,0) q[9];
cx q[1], q[7];
rx(3.8978138513725322) q[7];
cx q[9], q[8];
cx q[1], q[6];
cx q[5], q[6];
cx q[1], q[4];
cx q[7], q[0];
cx q[8], q[7];
cx q[0], q[7];
rx(3.755928975461476) q[0];
cx q[8], q[6];
U(0,2.971557384998350

░  ┌────────────┐                     ┌───┐                      »
 q_0: ──────░──┤ Rz(2.2874) ├─────────────────────┤ X ├──────────────────────»
            ░ ┌┴────────────┤┌───────────────┐    └─┬─┘     ┌───────────────┐»
 q_1: ──────░─┤ Rz(0.58487) ├┤ U(0,5.9533,0) ├──────┼───────┤ U(0,1.7731,0) ├»
            ░ └─────────────┘└─┬────────────┬┘      │       └───────────────┘»
 q_2: ──────░────────■─────────┤ Rx(4.0909) ├───────┼────────────────────────»
            ░        │         └────────────┘       │                        »
 q_3: ──────░────────┼──────────────────────────────■───────────────■────────»
            ░        │                        ┌────────────┐        │        »
 q_4: ──────░────────┼───────────────■────────┤ Rx(1.0187) ├────────┼────────»
      ┌───┐ ░        │               │        ├────────────┤        │        »
 q_5: ┤ X ├─░────────┼───────────────┼────────┤ Rz(6.1429) ├────────┼────────»
      ├───┤ ░        │               │        ├───────────┬┘        │        »
 q_6: ┤ X ├─░────────┼───────────────┼────────┤ Rz(0.591) ├─────────┼────────»
      ├───┤ ░        │             ┌─┴─┐      └───────────┘         │        »
 q_7: ┤ X ├─░────────┼─────────────┤ X ├────────────────────────────┼────────»
      ├───┤ ░      ┌─┴─┐           └───┘                          ┌─┴─┐      »
 q_8: ┤ X ├─░──────┤ X ├──────────────────────────────────────────┤ X ├──────»
      └───┘ ░  ┌───┴───┴────┐  ┌────────────┐                     └───┘      »
 q_9: ──────░──┤ Rx(4.5494) ├──┤ Rz(5.7293) ├────────────────────────────────»
            ░  └────────────┘  └────────────┘                                »
c: 10/═══════════════════════════════════════════════════════════════════════»
                                                                             »
«            ┌───┐                       ┌───┐┌───────────────┐┌──────────────┐»
« q_0: ──────┤ X ├───────────────────────┤ X ├┤ U(0,1.2855,0) ├┤ U(0,1.741,0) ├»
«            └─┬─┘        ┌────────────┐ └─┬─┘└───────────────┘└──────────────┘»
« q_1: ────────┼──────────┤ Ry(2.3365) ├───┼──────────────────────────■────────»
«              │          ├────────────┤   │                          │        »
« q_2: ────────■──────────┤ Rx(6.2063) ├───┼──────────■───────────────┼────────»
«      ┌───────────────┐  ├────────────┤   │          │               │        »
« q_3: ┤ U(0,1.7229,0) ├──┤ Ry(2.4703) ├───┼──────────┼───────────────┼────────»
«      └───────────────┘┌─┴────────────┴┐  │          │               │        »
« q_4: ────────■────────┤ U(0,2.4485,0) ├──┼──────────┼───────────────┼────────»
«              │        └───────────────┘  │        ┌─┴─┐             │        »
« q_5: ────────┼───────────────────────────┼────────┤ X ├─────────────┼────────»
«              │                           │        └───┘             │        »
« q_6: ────────┼───────────────────────────■──────────────────────────┼────────»
«              │                                                    ┌─┴─┐      »
« q_7: ────────┼────────────────■───────────────────────────────────┤ X ├──────»
«              │                │                                   └───┘      »
« q_8: ────────┼────────────────┼──────────────────────────────────────────────»
«            ┌─┴─┐            ┌─┴─┐                                            »
« q_9: ──────┤ X ├────────────┤ X ├────────────────────────────────────────────»
«            └───┘            └───┘                                            »
«c: 10/════════════════════════════════════════════════════════════════════════»
«                                                                              »
«                                             ┌───┐          ┌────────────┐»
« q_0: ───────────────────────────────────────┤ X ├───────■──┤ Rx(3.7559) ├»
«                                             └─┬─┘       │  └────────────┘»
« q_1: ─────────────────────────■──────────■────┼─────────┼────────────────»
«      ┌───────────────┐

In [48]:
from abc import ABC
from abc import abstractmethod

from qiskit import QuantumCircuit
from qiskit import QuantumCircuit, Aer, execute

import cirq
from cirq import Simulator
from cirq.contrib.qasm_import import circuit_from_qasm
from cirq.ops.measurement_gate import MeasurementGate


# CIRQ: supporting function
def get_all_measurement_keys(circuit):
    all_ops = list(circuit.findall_operations_with_gate_type(MeasurementGate))
    return sorted([e[2].key for e in all_ops])


class Circuit(ABC):
    
    def __init__(self, repetitions=1024):
        self.result = None
        self.circuit = None
        self.repetitions = repetitions
    
    @abstractmethod
    def from_qasm(self, qasm_string):
        pass
    
    @abstractmethod
    def execute(self, classical_input=None):
        pass
    
    @abstractmethod
    def draw(self):
        pass
    
    def get_result(self):
        return self.result    
    
class QiskitCircuit(Circuit):
    
    def __init__(self, repetitions=1024):
        super(QiskitCircuit, self).__init__(repetitions)
        self.platform_name = 'qiskit'
        self.simulator = Aer.get_backend("aer_simulator") # aer_simulator | qasm_simulator
    
    def from_qasm(self, qasm_string):
        self.circuit = QuantumCircuit.from_qasm_str(qasm_string)
    
    def execute(self, classical_input=None):
        job = execute(self.circuit, self.simulator)
        job_result = job.result()
        self.result = dict(job_result.get_counts())
    
    def draw(self):
        print(self.circuit.draw())
    
class CirqCircuit(Circuit):
    
    def __init__(self, repetitions=1024):
        super(CirqCircuit, self).__init__(repetitions)
        self.platform_name = 'cirq'
        self.simulator = Simulator()
    
    def from_qasm(self, qasm_string):
        # remember to add the QASM header
        
        chunks = qasm_string.split("barrier q;")
        preface = "\n".join(qasm_string.split("\n")[:5])
        # preface contains standatd lib and registers 
        # print(preface)
        
        # extract the input preparation part (before the 1st barrier)
        # create it with circuit_from_qasm(chunk_input)
        chunk_input = chunks[0]
        
        # extract the central part (between barriers)
        # create it with circuit_from_qasm(chunk_core)
        chunk_core = preface + chunks[1]
        
        # extract the last part (after 2nd barrier)
        # create it with circuit_from_qasm(chunk_measurement)
        chunk_measurement = preface + chunks[2]
        
        # glue the three chunks making sure they are executed in different moments
        # hint: use append
        
        self.circuit = circuit_from_qasm(chunk_input) + circuit_from_qasm(chunk_core) + circuit_from_qasm(chunk_measurement)
    
    def execute(self, classical_input=None):
        samples = self.simulator.run(self.circuit, repetitions=self.repetitions)
        all_keys = get_all_measurement_keys(self.circuit)
        counter = samples.multi_measurement_histogram(keys=all_keys)
        self.result = {
            "".join([str(d) for d in e][::-1]): int(v) 
            for (e, v) in counter.items()
        }
    
    def draw(self):
        print(self.circuit.to_text_diagram())

In [49]:
print(random_circuit_qasm_str)


OPENQASM 2.0;
include "qelib1.inc";
qreg q[10];
creg c[10];
x q[5];
x q[6];
x q[7];
x q[8];
barrier q;
cx q[1], q[4];
cx q[8], q[5];
cx q[0], q[6];
cx q[7], q[8];
rx(4.944004304573403) q[1];
cx q[2], q[5];
U(0,5.689917531808883,0) q[2];
cx q[0], q[7];
cx q[3], q[1];
cx q[2], q[7];
ry(4.551675688481526) q[9];
ry(5.547848144046363) q[6];
cx q[3], q[7];
cx q[9], q[4];
cx q[7], q[2];
U(0,4.189604350404568,0) q[9];
cx q[2], q[5];
rz(0.6559862796950657) q[1];
cx q[5], q[8];
cx q[2], q[7];
rz(4.932728082891125) q[8];
cx q[3], q[9];
U(0,4.456457279568166,0) q[6];
U(0,0.8599598304952863,0) q[4];
ry(4.380675085056772) q[9];
rz(0.6060958728071278) q[4];
cx q[9], q[4];
cx q[9], q[4];
rz(0.6390142944918692) q[5];
ry(0.21826392552498752) q[9];
rz(2.4747344635395274) q[2];
cx q[8], q[4];
rx(0.3072925876811064) q[8];
cx q[0], q[7];
rx(3.1323746111030624) q[8];
cx q[8], q[2];
U(0,1.0538199418251537,0) q[2];
rx(0.22878163741099564) q[0];
rx(0.058764360147966814) q[2];
cx q[4], q[3];
cx q[5], q[7];
ry(0

In [50]:
# Preselection (optional)
# IDEA: maybe we should keep only programs which output a subset of states and not superpositions
# https://quantumcomputing.stackexchange.com/a/14398/18188
# how-many-shots-should-one-take-to-get-a-reliable-estimate-in-a-quantum-program

platforms = [QiskitCircuit(), CirqCircuit()]
platform_result_dfs = []

# for each platform in (qiskit, cirq, etc)
for p in platforms:
    print(p.platform_name)

    # create a circuit object form the qasm file
    p.from_qasm(random_circuit_qasm_str)
    
    # perform 30 evaluations [N_EVALUATION]
    data = []
    for i in range(N_EVALUATIONS):
        # run the circuit for 1024 or 8096 shots
        p.execute()
        data.append(p.get_result())
    
    # save the statistics for each combination to a dataframe
    df = pd.DataFrame.from_records(data)
    platform_result_dfs.append(df)

for df in platform_result_dfs:
    display(df.head())

qiskit
cirq


,1001100010,0011101000,1000011110,1100010100,0111011000,1110001000,0000000100,0111010010,0101100100,1000000111,...,1110001100,1101110101,0011011011,1000110100,0101110011,0110001100,1100101101,1111111111,1110111011,0010010101
0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,3.0,2.0,1.0,NaN,3.0,1.0,NaN,6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,1.0,1.0,3.0,3.0,3.0,1.0,NaN,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,1.0,1.0,6.0,2.0,2.0,3.0,NaN,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,NaN,2.0,5.0,1.0,1.0,1.0,1.0,3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,1000001010,1110000101,0110010010,0011001010,1001011110,1101001100,0011001000,0100100100,1101101100,1000000000,...,0010001110,1110000111,0000000111,0000101101,1111101111,0011010101,1111001111,0011100011,1100101101,1001110001
0,5.0,1.0,3.0,15,4.0,5,13,2.0,1.0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.0,NaN,1.0,8,6.0,3,19,2.0,NaN,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,NaN,NaN,11,2.0,4,21,3.0,NaN,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,1.0,9,7.0,4,18,2.0,NaN,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,NaN,6,1.0,2,14,2.0,NaN,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
df_qiskit, df_cirq = platform_result_dfs 

In [66]:

m = df_qiskit.fillna(0).to_numpy()
solution_cum_freq = m.sum(axis=0)
most_frequent_solutions = np.argsort(solution_cum_freq)[::-1][:FIRST_K]
relevant_solutions = df_qiskit.columns[most_frequent_solutions]
freqs = solution_cum_freq[most_frequent_solutions]

In [73]:
df_relevant_qiskit = df_qiskit[relevant_solutions]
df_relevant_qiskit.head(3)

,1101001000,0011001000,0011100100,1101100100,0001001010
0,22,17,13,25,7
1,18,7,16,17,16
2,20,18,16,16,20


In [74]:
df_relevant_cirq = df_cirq[relevant_solutions]
df_relevant_cirq.head(3)

,1101001000,0011001000,0011100100,1101100100,0001001010
0,13,13,15,7,13
1,20,19,30,10,12
2,22,21,17,17,13


In [76]:
    
# compare the distribution of the occurrences of each basis state (e.g. 0000)
# for cirq (state: 0000): [512, 493, 502, etc ... until N_EVALUATION]
# for qiskit (state: 0000): [400, 475, 480, etc ... until N_EVALUATION]
# check if the two sequences come from the same distribution with a test
from scipy.stats import ttest_ind


def score(df_master, df_slave, first_k=TOP_K_SOLUTION_TO_COMPARE):
    # get the top k occurring solutions according to the master
    m = df_master.fillna(0).to_numpy()
    solution_cum_freq = m.sum(axis=0)
    most_frequent_solutions = np.argsort(solution_cum_freq)[::-1][:first_k]
    relevant_solutions = df_master.columns[most_frequent_solutions]
    freqs = solution_cum_freq[most_frequent_solutions]
    # compute p-values
    p_values = []
    df_master = df_master.fillna(0)
    df_slave = df_slave.fillna(0)
    for c in relevant_solutions:
        try:
            stat, p = ttest_ind(df_master[c], df_slave[c])
        except:
            if c not in df_master.columns:
                print(f"{c} column was missing from result of platform master")
                df_master[c] = 0
            if c not in df_slave.columns:
                print(f"{c} column was missing from result of platform slave")
                df_slave[c] = 0
            stat, p = ttest_ind(df_master[c], df_slave[c])    
        p_values.append(p)
    return min(p_values)
    
score(df_cirq, df_qiskit)

0.06832462912566448

In [ ]:
# if yes... then bug discovered!

# next steps:
    
    # which platform is buggy: cirq or qiskit?
    
    # where was the mistake in that platform?
    
        # delta debugging (style)
        
        # - remove operations
        # - track the function called with cProfile
        # - continue until the removed operation cause the Cirq an Qiskit result 
        # to agree again

## Searching Loop

In [87]:
N_TRIALS = 10
P_VALUE_THRESHOLD = 0.01

In [104]:
np.random.seed(SEED_NP)
  
platforms = [QiskitCircuit(), CirqCircuit()]

for t in range(1): #tqdm(range(N_TRIALS)):
    # create new circuit
    random_circuit_qasm_str = encoding_to_circuit(
        # FROM THE FUTURE GA SELECTION
        encoding=best_candidate,
        #encoding=random_circuit_encoding(), 
        n_qubits=N_QUBITS, 
        classical_input=CLASSICAL_INPUT)
    
    platform_result_dfs = []

    # for each platform in (qiskit, cirq, etc)
    for p in platforms:
        #print(p.platform_name)

        # create a circuit object form the qasm file
        p.from_qasm(random_circuit_qasm_str)

        # perform n evlauations of the same circuit with the same platform
        data = []
        for i in range(N_EVALUATIONS):
            # run the circuit for 1024 or 8096 shots
            p.execute()
            data.append(p.get_result())

        # save the statistics for each combination to a dataframe
        df = pd.DataFrame.from_records(data)
        platform_result_dfs.append(df)
        
    # compare results of two platforms
    df_qiskit, df_cirq = platform_result_dfs 
    
    lowest_p_value = score(df_cirq, df_qiskit)
    if True: #lowest_p_value < P_VALUE_THRESHOLD:
        print(f'EMERGENCY: lowest p_value: {lowest_p_value}')
        print(random_circuit_qasm_str)
        m = df_cirq.fillna(0).to_numpy()
        print(f"Total solutions appeared: {df_cirq.columns}")
        solution_cum_freq = m.sum(axis=0)
        most_frequent_solutions = np.argsort(solution_cum_freq)[::-1][:TOP_K_SOLUTION_TO_COMPARE]
        relevant_solutions = df_cirq.columns[most_frequent_solutions]
        display(df_cirq[relevant_solutions].head())
        display(df_qiskit[relevant_solutions].head())

EMERGENCY: lowest p_value: 0.012180700771626822

OPENQASM 2.0;
include "qelib1.inc";
qreg q[10];
creg c[10];
x q[3];
x q[4];
x q[5];
x q[6];
x q[7];
x q[8];
x q[9];
barrier q;
ry(3.824043218929409) q[0];
cx q[7], q[0];
cx q[4], q[8];
ry(0.4083814200344868) q[2];
cx q[4], q[2];
ry(1.722869882669675) q[8];
cx q[0], q[8];
ry(5.594175641533652) q[0];
rx(4.6326522017598935) q[3];
cx q[8], q[3];
barrier q;
measure q -> c;

Total solutions appeared: Index(['1011111100', '1111111101', '1111110101', '1111111100', '1111110100',
       '1011110100', '1011111101', '1011110101', '1111111000', '1011111001',
       '1111110001', '1111110000', '1011111000', '1011110000', '1111111001',
       '1011110001'],
      dtype='object')


,1111111100,1111110100,1011110100,1011111100,1011110101
0,185,162,144,110,104
1,180,153,141,96,117
2,185,157,128,108,116
3,180,185,119,110,107
4,201,155,138,127,98


,1111111100,1111110100,1011110100,1011111100,1011110101
0,188,153,126,110,103
1,178,171,116,94,109
2,181,178,115,112,96
3,190,174,110,114,106
4,187,171,138,114,112


## Identify circuits with few different solutions
Intuition: if the circuit is giving more peaked distributions, it is more realistic, indeed useful algorithms have solutions with a high probability of occurrence.

We can use the genetic algorith, framework to give select individuals (circuit) which are peaked in their output (aka there are some outputs which get very high occurrence)

In [93]:
PLATFORM = QiskitCircuit()
N_EVALUATIONS_GA = 10
N_OPS_GA = 10

def fitness_function(encoding):
    # create new circuit
    random_circuit_qasm_str = encoding_to_circuit(
        encoding=encoding, 
        n_qubits=N_QUBITS, 
        classical_input=CLASSICAL_INPUT)
    p = PLATFORM
    p.from_qasm(random_circuit_qasm_str)
    p.execute()
    # perform n evlauations of the same circuit with the same platform
    data = []
    for i in range(N_EVALUATIONS_GA):
        # run the circuit for 1024 or 8096 shots
        p.execute()
        data.append(p.get_result())
    # save the statistics for each combination to a dataframe
    df = pd.DataFrame.from_records(data)
    m = df_cirq.fillna(0).to_numpy()
    return max(m.sum(axis=0))

In [100]:
import random
from tqdm import tqdm
from deap import creator, base, tools, algorithms
# change for scoop
#from scoop import futures

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# original 
# toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("attr_bool", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=3*N_OPS_GA)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
# change for scoop
#toolbox.register("map",futures.map) # <--------------- overload the map function

def evalOneMax(individual):
    print("*")
    return fitness_function(encoding=individual),

toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

population = toolbox.population(n=30)

NGEN = 10
for gen in tqdm(range(NGEN)):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.1)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=len(population))
top10 = tools.selBest(population, k=10)

  0%|                                                                                                                                                       | 0/10 [00:00<?, ?it/s]

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*


 10%|██████████████▎                                                                                                                                | 1/10 [00:18<02:50, 18.92s/it]

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*


 20%|████████████████████████████▌                                                                                                                  | 2/10 [00:38<02:33, 19.25s/it]

...

 80%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                            | 8/10 [02:27<00:36, 18.02s/it]

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*


 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋              | 9/10 [02:45<00:18, 18.10s/it]

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [03:03<00:00, 18.32s/it]


In [101]:
best_candidate = top10[0]
random_circuit_qasm_str = encoding_to_circuit(
    encoding=best_candidate, 
    n_qubits=N_QUBITS, 
    classical_input=CLASSICAL_INPUT)
p = PLATFORM
p.from_qasm(random_circuit_qasm_str)
p.draw()

            ░  ┌───────────┐               ┌───┐     ┌────────────┐ ░ ┌─┐      »
 q_0: ──────░──┤ Ry(3.824) ├───────────────┤ X ├──■──┤ Ry(5.5942) ├─░─┤M├──────»
            ░  └───────────┘               └─┬─┘  │  └────────────┘ ░ └╥┘┌─┐   »
 q_1: ──────░────────────────────────────────┼────┼─────────────────░──╫─┤M├───»
            ░ ┌─────────────┐    ┌───┐       │    │                 ░  ║ └╥┘┌─┐»
 q_2: ──────░─┤ Ry(0.40838) ├────┤ X ├───────┼────┼─────────────────░──╫──╫─┤M├»
      ┌───┐ ░ └┬────────────┤    └─┬─┘       │    │      ┌───┐      ░  ║  ║ └╥┘»
 q_3: ┤ X ├─░──┤ Rx(4.6327) ├──────┼─────────┼────┼──────┤ X ├──────░──╫──╫──╫─»
      ├───┤ ░  └────────────┘      │         │    │      └─┬─┘      ░  ║  ║  ║ »
 q_4: ┤ X ├─░────────■─────────────■─────────┼────┼────────┼────────░──╫──╫──╫─»
      ├───┤ ░        │                       │    │        │        ░  ║  ║  ║ »
 q_5: ┤ X ├─░────────┼───────────────────────┼────┼────────┼────────░──╫──╫──╫─»
      ├───┤ ░        │      